In [41]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as pyo



def makechart():
    table=pd.read_csv('price.csv')
    table=table[table["Price"]>100]
    table["Date"]=pd.to_datetime(table["Date"])
    table["Street"]=table["Street"].str.replace("  "," ")
    table["last word"]=table["Street"].str.split().str[-1]
    

    freq=table["last word"].value_counts()
    table=table.join(freq,on="last word",rsuffix="_count")
    table.loc[table["count"]<50, "last word"] = "OTHER"
    table=table.sort_values(by="count",ascending=False)
    table["last word"]=table["last word"].astype(str)
    grouped=table[['last word','Price']].groupby('last word').agg(['count','mean','min','max','std','median'])
    grouped=grouped.reset_index()
    grouped.columns=["last word","count","mean","min","max","std","median"]
    grouped=grouped[grouped["last word"]!="nan"]

    grouped["maxdiff"]=grouped["max"]-grouped["median"]
    grouped["mindiff"]=grouped["median"]-grouped["min"]


    
    buttons = []
    for key in ["median", "mean", "count", "min", "max", "last word"]:
        
        if key=="last word":
            grouped=grouped.sort_values(by=key, ascending=False)
        else:
            grouped=grouped.sort_values(by=key, ascending=True)
        x_median = grouped["median"].tolist()
        x_mean = grouped["mean"].tolist()
        x_count = grouped["count"].tolist()
        x_min = grouped["min"].tolist()
        x_max = grouped["max"].tolist()
        x_last_word = grouped["last word"].tolist()
        y_words = grouped["last word"].tolist()

        buttons.append({"label": f"Sort by {key}","method": "update","args": [
            {"x": [x_median, x_mean, x_count, x_min, x_max, x_last_word], 
            "y": [y_words, y_words, y_words,y_words,y_words,y_words]},[0, 1, 2]]})
    
    #fig=px.scatter(grouped,x="min",y="last word",height=3300,width=1000,log_x=True)
    fig=px.scatter(grouped,x="median",y="last word",error_x="maxdiff",error_x_minus="mindiff",height=3300,width=1000,log_x=True)
    fig.add_trace(px.scatter(grouped,x="mean",y="last word",color_discrete_sequence=["#001a3a"]).data[0])

    fig.add_trace(px.bar(grouped,x="count",y="last word",color_discrete_sequence=["#97adc9"]).update_traces(width=0.5).data[0])
    

    fig.update_layout(xaxis_side="top")
    fig.update_layout(xaxis_title="Price",yaxis_title="Last word in street name",yaxis=dict(tickfont=dict(size=8)))
    fig.update_layout(margin=dict(t=0))

    fig.update_layout(updatemenus=[dict(
                buttons=buttons,
                direction="down",
                showactive=True)])
              
    fig.write_html("graph.html")
    fig.show()
makechart()

